In [ ]:
"""
--- Day 8: Handheld Halting ---
Your flight to the major airline hub reaches cruising altitude without incident. 
While you consider checking the in-flight menu for one of those drinks that come 
with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code 
(your puzzle input) of the device. You should be able to fix it, but first 
you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of 
text. Each instruction consists of an operation (acc, jmp, or nop) and an 
argument (a signed number like +4 or -20).

acc increases or decreases a single global value called the accumulator by the 
value given in the argument. For example, acc +7 would increase the accumulator 
by 7. The accumulator starts at 0. After an acc instruction, the instruction 
immediately below it is executed next.

jmp jumps to a new instruction relative to itself. The next instruction to execute 
is found using the argument as an offset from the jmp instruction; for example, 
jmp +2 would skip the next instruction, jmp +1 would continue to the instruction 
immediately below it, and jmp -20 would cause the instruction 20 lines above to be 
executed next.

nop stands for No OPeration - it does nothing. The instruction immediately below 
it is executed next.
For example, consider the following program:

nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
These instructions are visited in this order:

nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) 
and jmp +4 sets the next instruction to the other acc +1 near the bottom. After it 
increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction 
to the only acc +3. It sets the accumulator to 5, and jmp -3 causes the program to 
continue back at the first acc +1.

This is an infinite loop: with this sequence of jumps, the program will run forever. 
The moment the program tries to run any instruction a second time, you know it will 
never terminate.

Immediately before the program would run an instruction a second time, the value in 
the accumulator is 5.

Run your copy of the boot code. Immediately before any instruction is executed a second 
time, what value is in the accumulator?
"""

In [27]:
## puzzle input

inp_loc = './day_8_input.txt'

puzzle_input = []
with open(inp_loc, 'r') as txtfile:
    for line in txtfile:
        puzzle_input.append(line.rstrip().split(' '))

print('Items:', len(puzzle_input))

print(puzzle_input[0][0])

print(puzzle_input[-1])

Items: 634
acc
['jmp', '+1']


In [58]:
def boot_debug(instructions, start_idx, idx_exe, acc_val, stop_flag=0, error_code=0):
    """
    Works through instruction list keeping track of
    accumulator value. Sets stop flag once an instruction
    line is repeated
    """
    
    if start_idx in idx_exe:
        stop_flag = 1
        error_code = 1
        return start_idx, idx_exe, acc_val, stop_flag, error_code
    
    if start_idx == len(instructions):
        stop_flag = 1
        print("Boot Test Successful")
        return start_idx, idx_exe, acc_val, stop_flag, error_code
    
    if instructions[start_idx][0].lower() == 'nop':
        idx_exe.append(start_idx)
        start_idx += 1
    
    elif instructions[start_idx][0].lower() == 'acc':
        idx_exe.append(start_idx)
        acc_val += int(instructions[start_idx][1])
        start_idx += 1
    
    elif instructions[start_idx][0].lower() == 'jmp':
        idx_exe.append(start_idx)
        start_idx += int(instructions[start_idx][1])
    
    return start_idx, idx_exe, acc_val, stop_flag, error_code



idx_exe = []
start_idx = 0
acc_val = 0
stop_flag = 0

while stop_flag != 1:
    start_idx, idx_exe, acc_val, stop_flag, error_code = boot_debug(puzzle_input, start_idx, idx_exe, acc_val)
print(acc_val)

1749


In [ ]:
"""
--- Part Two ---
After some careful analysis, you believe that exactly one instruction is 
corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop 
is supposed to be a jmp. (No acc instructions were harmed in the corruption 
of this boot code.)

The program is supposed to terminate by attempting to execute an instruction 
immediately after the last instruction in the file. By changing exactly one 
jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:

nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
If you change the first instruction from nop +0 to jmp +0, it would create 
a single-instruction infinite loop, never leaving that instruction. If you 
change almost any of the jmp instructions, the program will still eventually
find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from jmp -4 to nop -4),
the program terminates! The instructions are visited in this order:

nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
After the last instruction (acc +6), the program terminates by attempting to 
run the instruction below the last instruction in the file. With this change, 
after the program terminates, the accumulator contains the value 8 
(acc +1, acc +1, acc +6).

Fix the program so that it terminates normally by changing exactly one jmp 
(to nop) or nop (to jmp). What is the value of the accumulator after the
program terminates?
"""

In [22]:
import copy

In [16]:
# create lists with index locations of all `jmp` and `nop` locations

jmps = []
nops = []

for i in range(len(puzzle_input)):
    if puzzle_input[i][0].lower() == 'jmp':
        jmps.append(i)
    if puzzle_input[i][0].lower() == 'nop':
        nops.append(i)
        
print(len(jmps))
print(len(nops))

221
62


In [63]:
# loop through each index location list
# create a temporary altered instructions list
# append error code to a list, find location where error DIDN't occur (0)



jmp_errors = []
jmp_acc_vals = []
for jmp in jmps:
    tmp_instr = copy.deepcopy(puzzle_input)
    tmp_instr[jmp][0] = 'nop'
    
    idx_exe = []
    start_idx = 0
    acc_val = 0
    stop_flag = 0
    
    while stop_flag != 1:
        start_idx, idx_exe, acc_val, stop_flag, error_code = boot_debug(tmp_instr, start_idx, idx_exe, acc_val)
    jmp_errors.append(error_code)
    jmp_acc_vals.append(acc_val)
    


    
nop_errors = []
nop_acc_vals = []
for nop in nops:
    tmp_instr = copy.deepcopy(puzzle_input)
    tmp_instr[nop][0] = 'jmp'
    
    idx_exe = []
    start_idx = 0
    acc_val = 0
    stop_flag = 0
    
    while stop_flag != 1:
        start_idx, idx_exe, acc_val, stop_flag, error_code = boot_debug(tmp_instr, start_idx, idx_exe, acc_val)
    jmp_errors.append(error_code)
    nop_acc_vals.append(acc_val)


jmp_err_idx = [idx for idx, ele in enumerate(jmp_errors) if ele == 0]
nop_err_idx = [idx for idx, ele in enumerate(nop_errors) if ele == 0]

print()
    
if len(jmp_err_idx) != 0:
    print("Error found on jmp command at index {}".format(jmps[jmp_err_idx[0]]))
    print("\nacc value at boot termination: {}".format(jmp_acc_vals[jmp_err_idx[0]]))
if len(nop_err_idx) != 0:
    print("Error found on nop command at index {}".format(nops[nop_err_idx[0]]))
    print("\nacc value at boot termination: {}".format(nop_acc_vals[nop_err_idx[0]]))
    

Boot Test Successful

Error found on jmp command at index 432

acc value at boot termination: 515
